In [1]:
!apt install aptitude swig > /dev/null
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null
!pip install mecab-python3==0.996.3 > /dev/null
!pip install pykakasi > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null



Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11.6M  100 11.6M    0     0  4673k      0  0:00:02  0:00:02 --:--:-- 7092k
./../../libexec/iconv_euc_to_utf8.sh ./Noun.name.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Filler.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Adverb.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.place.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Others.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Verb.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.adjv.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Nou

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7" -O /content/dataset.zip && rm -rf /tmp/cookies.txt
!unzip -q /content/dataset.zip -d /content

--2021-03-16 00:11:43--  https://docs.google.com/uc?export=download&confirm=j6VK&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7
Resolving docs.google.com (docs.google.com)... 142.250.98.100, 142.250.98.139, 142.250.98.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.98.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-88-docs.googleusercontent.com/docs/securesc/p0bm70vejiv989d6c3g5l6g2jiubn0oa/bfiibotmql7ovf9dof2ce2vvpr8vvdu6/1615853475000/03816308716299652124/12102026523754886384Z/1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7?e=download [following]
--2021-03-16 00:11:44--  https://doc-0k-88-docs.googleusercontent.com/docs/securesc/p0bm70vejiv989d6c3g5l6g2jiubn0oa/bfiibotmql7ovf9dof2ce2vvpr8vvdu6/1615853475000/03816308716299652124/12102026523754886384Z/1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7?e=download
Resolving doc-0k-88-docs.googleusercontent.com (doc-0k-88-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting

In [3]:
import subprocess
import MeCab

cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))

In [65]:
import os
import json

json_file_path = '/content/jsut_ver1.1/basic5000/json/BASIC5000_0004.json'
timestamps = []

with open(json_file_path, 'r', encoding='UTF-8') as fj:
  content = fj.read()
  content = json.loads(content)
  transcript = content['results'][0]['alternatives'][0]['transcript']

In [66]:
print(transcript)
print(m.parse(transcript))

一週間 して その ニュース は 本当 に なった 
一週間	名詞,固有名詞,一般,*,*,*,一週間,イッシュウカン,イッシューカン
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
その	連体詞,*,*,*,*,*,その,ソノ,ソノ
ニュース	名詞,一般,*,*,*,*,ニュース,ニュース,ニュース
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
本当	名詞,一般,*,*,*,*,本当,ホントウ,ホントー
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
なっ	動詞,自立,*,*,五段・ラ行,連用タ接続,なる,ナッ,ナッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
EOS



In [67]:
node = m.parseToNode(transcript).next
transcript_splited = transcript.split(' ')

term_idx = []
terms = []
term_buf = ''

index = 0
while node:

    # 単語
    term = node.surface
    term_buf = term_buf + term

    # 品詞
    pos = node.feature.split(',')[0]

    # transcriptより細かい粒度で分割された時はバッファにためる
    if term_buf != transcript_splited[index]:
      node = node.next
      continue

    # もし品詞が条件と一致してたら
    if pos == '助詞':
      # 助詞が連続している場合は最後のものを使用する
        if len(term_idx) !=0 and term_idx[-1] == index-1:
          term_idx = term_idx[:len(term_idx)-1]
          terms = terms[:len(terms)-1]
        term_idx.append(index)
        terms.append(term_buf)

    term_buf = ''
    node = node.next
    index = index+1

In [68]:
print(term_idx)
print(terms)

[1, 4, 6]
['して', 'は', 'に']
